#### **Tools in Langchain**  

Documentation - https://docs.langchain.com/oss/python/integrations/tools  
There are a lot of tools & toolkits present which we can source through the documentation.

Tool is just a Python function packaged in a way that the LLM can understand and call it when needed.  

**Tool Workflow-**
LLM reasons what needs to be done.  
LLM calls a tool (i.e., a Python function).  
Tool performs task (like API call, DB query, code execution).  
LLM receives the result and responds.

##### **Built-In Tools**

**DuckDuckGo Search-** Used to search on the internet

In [13]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("Top news in India Today related to stock market")
print(results)

... share in the Indian share market ! MRF has never split its stock ... You need a Demat and trading account to invest your money in the stock market . ... indices are trading on a positive note today with stocks ... Now track the biggest movers of the stock market using stocks to watch today section. Sectoral indices are trading mixed today with stocks in ... Now track the biggest movers of the stock market using stocks to watch today section. There are many stock market news apps in India , therefore, it might be a little difficult to choose. ... to get updates about the Indian stock market , ... Description：Reuters India offers top Business and market news updates covering Indian Stock market news , finance, investment and more news ...


**Shell Tool-** Used to run shell commands

In [14]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

result = shell_tool.invoke("ls")
print(result)

Executing command:
 ls
tools_basics.ipynb



c:\Users\Aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [15]:
# To know about the tool function
print(shell_tool.name)
print(shell_tool.description)
print(shell_tool.args)

terminal
Run shell commands on this Windows machine.
{'commands': {'anyOf': [{'type': 'string'}, {'items': {'type': 'string'}, 'type': 'array'}], 'description': 'List of shell commands to run. Deserialized using json.loads', 'title': 'Commands'}}


#### **Custom Tools**  
Custom Tools in LangChain are functions that you build when built-in tools are not enough for your application use case.  

**Use a Custom Tool when:**  
You need to call your own API inside a LangChain agent (e.g. hotel booking APIs).  
You have some custom business logic unique to your application.  
You want your LLM to interact with your own database, product, or application.

##### **Method 1 - @tool method**

In [16]:
from langchain_core.tools import tool

@tool
def multiply(a: float, b:float) -> float:  # Comments added so that the tool can be used in the agent (LLM knows what this tool does)
    """Multiply two numbers together.  
    
    Args:
        a (float): The first number.
        b (float): The second number.
    Returns: """
    return a * b


result = multiply.invoke({"a":1.1, "b":2.5})
print(result)

2.75


In [17]:
# To know about the tool function
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers together.  

    Args:
        a (float): The first number.
        b (float): The second number.
    Returns:
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


##### **Method 2- Using Structured Tool**

In [18]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class MultiplyInput(BaseModel):
    a: float = Field(required=True, description="The first no.")
    b: float = Field(required=True, description="The second no.")

def multiply(a: float, b:float) -> float:
    return a*b

multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="Multiply two numbers together",
    args_schema = MultiplyInput
)

result = multiply_tool.invoke({"a":1.1, "b":2.5})
print(result)

2.75


In [19]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two numbers together
{'a': {'description': 'The first no.', 'required': True, 'title': 'A', 'type': 'number'}, 'b': {'description': 'The second no.', 'required': True, 'title': 'B', 'type': 'number'}}


##### **Method 3 - Using BaseTool Class**  
@tool and Structured Tool are derived from this only

In [20]:
from langchain_core.tools import BaseTool
from typing import Type

# arg schema using Pydantic
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first no.")
    b: int = Field(required=True, description="The second no.")

class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers together"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a*b

multiply_tool = MultiplyTool()
result = multiply_tool.invoke({"a":10, "b":20})
print(result)

200


#### **Toolkit**
A Toolkit is a collection of related tools grouped together to serve a common purpose, allowing for: Better organization, Easier reuse across multiple applications, Cleaner code architecture

**Use a toolkit when:** You have multiple tools that are related in functionality.

Example: Tool to upload a file to Google Drive, Tool to read from Google Drive, Tool to search Google Drive files → These can be grouped into a GoogleDriveToolkit

In [21]:
from langchain_core.tools import tool

# Custom Tool
@tool
def add(a: int, b:int) -> int:
    """Add two numbers together"""
    return a+b

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers together"""
    return a*b


class MathToolkit:
    def get_tools(self):
        return [add, multiply]

toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print("Tool Name:", tool.name, "->", "Tool Description:", tool.description)

Tool Name: add -> Tool Description: Add two numbers together
Tool Name: multiply -> Tool Description: Multiply two numbers together


#### **Connecting Tools with an LLM**

**Tool Binding**  
Tool binding is the process of connecting or registering tools (functions) with an LLM so that the model knows: What tools are available, What each tool does (via description) & What input format the tool expects (via schema).

**Tool Calling**  
Tool Calling happens when the LLM realizes it needs a tool (e.g., for a math calculation), and generates a structured request containing the tool's name and the inputs it should be called with.

**Tool Execution**  
Tool Execution is done by LangChain or the developer, not by the LLM—it takes the tool name and inputs suggested by the LLM and actually runs the tool to get the result.

**Tool Message**  
A Tool Message is the final result returned after a tool is executed, and it is passed back to the LLM so it can use that output to continue the conversation meaningfully.

In [33]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

load_dotenv()

True

In [ ]:
@tool
def multiply(a:int, b:int) -> int:
    """Given 2 nos. a & b, this tools returns their product"""
    return a*b

model = ChatGroq(model="llama-3.3-70b-versatile")

model_with_tools = model.bind_tools([multiply])  # Tool Binding
model_with_tools.invoke('Hi How are you?').content

"I'm doing well, thanks for asking. Is there something I can help you with or would you like to chat?"

In [ ]:
query = HumanMessage("Can you multiply 3 with 100")

messages = [query]  # Maintains chat History
messages


[HumanMessage(content='Can you multiply 3 with 100', additional_kwargs={}, response_metadata={})]

In [38]:
result = model_with_tools.invoke(messages)
print(result.tool_calls)  # Tool Calling

messages.append(result)

[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'eet8k2012', 'type': 'tool_call'}]


In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])
print(tool_result)

messages.append(tool_result)
print(messages)  # Stores the Tool Messages

content='300' name='multiply' tool_call_id='eet8k2012'
[HumanMessage(content='Can you multiply 3 with 100', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'eet8k2012', 'function': {'arguments': '{"a":3,"b":100}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 245, 'total_tokens': 264, 'completion_time': 0.053806031, 'prompt_time': 0.012151477, 'queue_time': 0.053593452, 'total_time': 0.065957508}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6b1d6421-601a-49de-a383-567c415bcc5a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'eet8k2012', 'type': 'tool_call'}], usage_metadata={'input_tokens': 245, 'output_tokens': 19, 'total_tokens': 264}), ToolMessage(content='300', name='multiply'

In [41]:
model_with_tools.invoke(messages).content

'The result of multiplying 3 by 100 is 300.'